In [1]:
from datetime import date
import ast
import pandas as pd

pd.set_option('display.max_columns', None)

filter_label = catalog.load('params:openaire_fetch_options.filter_label')
df = catalog.load('raw/openaire/researchproduct_dev#parquet')

[02/20/25 11:42:12] INFO     Loading data from params:openaire_fetch_options.filter_label       ]8;id=446801;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=905290;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from raw/openaire/researchproduct_dev#parquet         ]8;id=450496;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=306399;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (ParquetDataset)...                                                                   

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   author              300 non-null    object
 1   openAccessColor     44 non-null     object
 2   publiclyFunded      285 non-null    object
 3   type                300 non-null    object
 4   language            300 non-null    object
 5   subjects            299 non-null    object
 6   mainTitle           300 non-null    object
 7   description         294 non-null    object
 8   publicationDate     300 non-null    object
 9   publisher           129 non-null    object
 10  source              82 non-null     object
 11  format              283 non-null    object
 12  bestAccessRight     224 non-null    object
 13  container           44 non-null     object
 14  id                  300 non-null    object
 15  originalId          300 non-null    object
 16  indicators          300 no

In [3]:
df.columns


Index(['author', 'openAccessColor', 'publiclyFunded', 'type', 'language',
       'subjects', 'mainTitle', 'description', 'publicationDate', 'publisher',
       'source', 'format', 'bestAccessRight', 'container', 'id', 'originalId',
       'indicators', 'instance', 'isGreen', 'isInDiamondJournal',
       'contributor', 'pid', 'country', 'contactPerson', 'filter_applied'],
      dtype='object')

## Paso 1: Convierto tipos y selecciono columnas con cardinalidad 1 con respecto a cada research product
+ info en https://graph.openaire.eu/docs/data-model/entities/research-product

In [4]:
def land_openaire_researchproduct(filter_value, df: pd.DataFrame)-> pd.DataFrame:

    df = df.convert_dtypes()

    expected_columns = [
        'filter_applied',
        'author',
        'openAccessColor',
        'publiclyFunded',
        'type',
        'language',
        'country',
        'subjects',
        'mainTitle',
        'description',
        'publicationDate',
        'format',
        'bestAccessRight',
        'id',
        'originalId',
        'indicators',
        'instance',
        'isGreen',
        'isInDiamondJournal',
        'publisher',
        'source',
        'container',
        'contributor',
        'contactPerson',
        'coverage',
        'pid',
        'contactPerson',
        'embargoEndDate',
    ]

    # Agregar columnas faltantes con NaN
    for col in expected_columns:
        if col not in df.columns:
            df[col] = pd.NA

    df_researchproduct = df[expected_columns].copy()
    df.reset_index(drop=True, inplace=True)

    # language
    df_researchproduct['language_code'] = df_researchproduct['language'].apply(lambda x: x['code'])
    df_researchproduct['language_label'] = df_researchproduct['language'].apply(lambda x: x['label'])

    ## bestAccessRight
    df_researchproduct['bestAccessRight_label'] = df['bestAccessRight'].apply(lambda x: x['label'] if x else None)
    df_researchproduct['bestAccessRight_scheme'] = df['bestAccessRight'].apply(lambda x: x['scheme'] if x else None)

    ## indicators
    df_indicators = pd.json_normalize(df['indicators']).reset_index(drop=True)
    
    indicators_expected_columns = [
        "citationImpact.citationClass",
        "citationImpact.citationCount",
        "citationImpact.impulse",
        "citationImpact.impulseClass",
        "citationImpact.influence",
        "citationImpact.influenceClass",
        "citationImpact.popularity",
        "citationImpact.popularityClass",
        "usageCounts.downloads",
        "usageCounts.views",
    ]

    # Agregar columnas para indicators y faltantes con NaN
    for col in indicators_expected_columns:
        if col not in df_indicators.columns:
            df_indicators[col] = pd.NA

    df_researchproduct = pd.concat([df_researchproduct.drop(columns=['indicators']).reset_index(drop=True), df_indicators], axis=1)

    ## author
    df_researchproduct2author = df.explode('author').reset_index(drop=True)
    df_researchproduct2author = df_researchproduct2author[['id','author']]
    df_authors = pd.json_normalize(df_researchproduct2author['author']).reset_index(drop=True)
    df_researchproduct2author = pd.concat([df_researchproduct2author.drop(columns=['author']), df_authors], axis=1)

    ## originalId
    df_researchproduct2originalId = df.explode('originalId').reset_index(drop=True)
    df_researchproduct2originalId = df_researchproduct2originalId[['id','originalId']]

    # TODO country

    ## subjects
    df_researchproduct2subject = df.explode('subjects').reset_index(drop=True)
    df_researchproduct2subject = df_researchproduct2subject[['id','subjects']]
    df_subjects = pd.json_normalize(df_researchproduct2subject['subjects']).reset_index(drop=True)
    df_researchproduct2subject = pd.concat([df_researchproduct2subject.drop(columns=['subjects']), df_subjects], axis=1)

    # TODO description

    # TODO format

    # TODO instance

    # TODO source

    # TODO container
    
    # TODO contributor
    
    # TODO contactPerson

    # TODO coverage
    
    # pid
    df_researchproduct2pid = df.explode('pid').reset_index(drop=True)
    df_researchproduct2pid = df_researchproduct2pid[['id','pid']]
    df_pid = pd.json_normalize(df_researchproduct2pid['pid']).reset_index(drop=True)
    df_researchproduct2pid = pd.concat([df_researchproduct2pid.drop(columns=['pid']), df_pid], axis=1)

    # url
    df_researchproduct2instance = df.explode('instance').reset_index(drop=True)
    df_researchproduct2instance = df_researchproduct2instance[['id','instance']]
    df_instance = pd.json_normalize(df_researchproduct2instance['instance']).reset_index(drop=True)
    df_researchproduct2instance = pd.concat([df_researchproduct2instance.drop(columns=['instance']), df_instance], axis=1)
    df_researchproduct2url = df_researchproduct2instance[['id','url']]
    df_researchproduct2url = df_researchproduct2url.explode('url')

    ## drop de columnas procesadas en otros df
    df_researchproduct.drop(columns=[
        'author', 'country', 'subjects','bestAccessRight', 
        'language', 'format', 'instance', 'originalId', 
        'container', 'source', 'pid', 'description',
        'contributor', 'contactPerson', 'coverage'
        ], inplace=True)

    df_researchproduct['load_datetime'] = date.today()
    df_researchproduct2originalId['load_datetime'] = date.today()
    df_researchproduct2author['load_datetime'] = date.today()
    df_researchproduct2subject['load_datetime'] = date.today()
    df_researchproduct2pid['load_datetime'] = date.today()
    df_researchproduct2url['load_datetime'] = date.today()

    return df_researchproduct, df_researchproduct2originalId, df_researchproduct2author, df_researchproduct2subject, df_researchproduct2pid, df_researchproduct2url


In [5]:
df_researchproduct, df_researchproduct2originalId, df_researchproduct2author, df_researchproduct2subject, df_researchproduct2pid, df_researchproduct2url = land_openaire_researchproduct(filter_label, df)

In [6]:
df_researchproduct

,filter_applied,openAccessColor,publiclyFunded,type,mainTitle,publicationDate,id,isGreen,isInDiamondJournal,publisher,embargoEndDate,language_code,language_label,bestAccessRight_label,bestAccessRight_scheme,citationImpact.citationClass,citationImpact.citationCount,citationImpact.impulse,citationImpact.impulseClass,citationImpact.influence,citationImpact.influenceClass,citationImpact.popularity,citationImpact.popularityClass,usageCounts.downloads,usageCounts.views,load_datetime
0,is_unlp,gold,False,publication,Variability in the growth rates of Saanen kids...,2023-12-18,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,False,False,"Universidad Nacional Mayor de San Marcos, Facu...",<NA>,spa,Spanish; Castilian,OPEN,http://vocabularies.coar-repositories.org/docu...,C5,0.0,0.0,C5,2.841867e-09,C5,2.885067e-09,C5,<NA>,<NA>,2025-02-20
1,is_unlp,<NA>,False,publication,Estudio de la viabilidad del algoritmo super-t...,2013-01-01,RECOLECTA___::24cb4438d1afe299e63cfdea4a31911f,True,False,<NA>,<NA>,Español,Español,OPEN,http://vocabularies.coar-repositories.org/docu...,C5,0.0,0.0,C5,2.841867e-09,C5,7.596338e-10,C5,<NA>,<NA>,2025-02-20
2,is_unlp,<NA>,False,publication,The Three Hundred project: The gas disruption ...,2021-03-01,core_ac_uk__::a391293fe8bd6c9c6c203ee15e8c2e8f,True,False,Oxford University Press (OUP),<NA>,eng,English,OPEN,http://vocabularies.coar-repositories.org/docu...,C5,0.0,0.0,C5,2.841867e-09,C5,2.155773e-09,C5,<NA>,<NA>,2025-02-20
3,is_unlp,<NA>,False,publication,Evolución del sistema de gestión de incidentes...,2023-06-23,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,True,False,<NA>,<NA>,esl/spa,Spanish,OPEN,http://vocabularies.coar-repositories.org/docu...,C5,0.0,0.0,C5,2.841867e-09,C5,2.885067e-09,C5,<NA>,<NA>,2025-02-20
4,is_unlp,gold,False,publication,Responsabilidad Social Corporativa en Empresas...,2013-12-01,dedup_wf_002::0000eb33b6be05d67799615dd117e5b2,False,False,Universidad Nacional de La Plata,<NA>,eng,English,OPEN,http://vocabularies.coar-repositories.org/docu...,C5,0.0,0.0,C5,2.841867e-09,C5,7.596338e-10,C5,<NA>,<NA>,2025-02-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,is_unlp,<NA>,False,publication,Respuesta inmunológica de bovinos vacunados si...,1978-01-01,dedup_wf_002::01504e314a4f352cc244d64c6cb72a79,True,False,<NA>,<NA>,esl/spa,Spanish,OPEN,http://vocabularies.coar-repositories.org/docu...,C5,0.0,0.0,C5,2.841867e-09,C5,2.239505e-10,C5,<NA>,<NA>,2025-02-20
296,is_unlp,<NA>,False,publication,"Workshop: ""Fluctuaciones macroeconómicas y cri...",2014-12-01,dedup_wf_002::0150d22e4775af3c7965c14921f4176f,True,False,<NA>,<NA>,esl/spa,Spanish,OPEN,http://vocabularies.coar-repositories.org/docu...,C5,0.0,0.0,C5,2.841867e-09,C5,8.529256e-10,C5,<NA>,<NA>,2025-02-20
297,is_unlp,<NA>,False,publication,Primeros apuntes ecológicos y herpetológicos s...,1970-01-01,dedup_wf_002::0152767df678ac928375ebd4d6079725,False,False,<NA>,<NA>,eng,English,None,None,C5,0.0,0.0,C5,2.841867e-09,C5,2.225151e-10,C5,<NA>,<NA>,2025-02-20
298,is_unlp,gold,False,publication,"“Serie del Encuentro”, primera colección del C...",2013-01-01,dedup_wf_002::0152976d6d89c7781aa7447192d99498,True,False,Universidad Nacional de La Plata,<NA>,esl/spa,Spanish,OPEN,http://vocabularies.coar-repositories.org/docu...,C5,0.0,0.0,C5,2.841867e-09,C5,7.596338e-10,C5,<NA>,<NA>,2025-02-20


In [7]:
df_researchproduct2originalId.head(5)

,id,originalId,load_datetime
0,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,oai:ojs.csi.unmsm:article/25240,2025-02-20
1,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,50|4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,2025-02-20
2,RECOLECTA___::24cb4438d1afe299e63cfdea4a31911f,50|RECOLECTA___::24cb4438d1afe299e63cfdea4a31911f,2025-02-20
3,RECOLECTA___::24cb4438d1afe299e63cfdea4a31911f,oai:digital.csic.es:10261/97737,2025-02-20
4,core_ac_uk__::a391293fe8bd6c9c6c203ee15e8c2e8f,oai:nottingham-repository.worktribe.com:5350191,2025-02-20


In [8]:
df_researchproduct2author.head(5)

,id,fullName,name,pid,rank,surname,pid.id.scheme,pid.id.value,pid.provenance,load_datetime
0,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,"Steffen, Kevin Denis",Kevin Denis,NaN,1,Steffen,NaN,NaN,NaN,2025-02-20
1,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,"Gortari Castillo, Lihuel",Lihuel,NaN,2,Gortari Castillo,NaN,NaN,NaN,2025-02-20
2,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,"Cordiviola, Carlos Ángel",Carlos Ángel,NaN,3,Cordiviola,NaN,NaN,NaN,2025-02-20
3,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,"Moré, Gastón",Gastón,NaN,4,Moré,NaN,NaN,NaN,2025-02-20
4,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,"Arias, Ruben Omar",Ruben Omar,NaN,5,Arias,NaN,NaN,NaN,2025-02-20


In [9]:
df_researchproduct2subject

,id,provenance,subject.scheme,subject.value,load_datetime
0,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,NaN,keyword,ganancia diaria de peso,2025-02-20
1,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,NaN,keyword,cabritos,2025-02-20
2,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,NaN,keyword,kids,2025-02-20
3,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,NaN,keyword,goats,2025-02-20
4,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,NaN,keyword,Saanen,2025-02-20
...,...,...,...,...,...
2263,dedup_wf_002::0152976d6d89c7781aa7447192d99498,NaN,keyword,Letras,2025-02-20
2264,dedup_wf_002::0154bda3fd593f174234f990829b4b34,NaN,keyword,Medios de información,2025-02-20
2265,dedup_wf_002::0154bda3fd593f174234f990829b4b34,NaN,keyword,Organización femenina,2025-02-20
2266,dedup_wf_002::0154bda3fd593f174234f990829b4b34,NaN,keyword,Comunicación Social,2025-02-20


In [10]:
df_researchproduct2pid

,id,scheme,value,load_datetime
0,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,NaN,NaN,2025-02-20
1,RECOLECTA___::24cb4438d1afe299e63cfdea4a31911f,handle,10261/97737,2025-02-20
2,core_ac_uk__::a391293fe8bd6c9c6c203ee15e8c2e8f,NaN,NaN,2025-02-20
3,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,NaN,NaN,2025-02-20
4,dedup_wf_002::0000eb33b6be05d67799615dd117e5b2,NaN,NaN,2025-02-20
...,...,...,...,...
321,dedup_wf_002::01504e314a4f352cc244d64c6cb72a79,NaN,NaN,2025-02-20
322,dedup_wf_002::0150d22e4775af3c7965c14921f4176f,NaN,NaN,2025-02-20
323,dedup_wf_002::0152767df678ac928375ebd4d6079725,NaN,NaN,2025-02-20
324,dedup_wf_002::0152976d6d89c7781aa7447192d99498,NaN,NaN,2025-02-20


In [11]:
df_researchproduct2url

,id,url,load_datetime
0,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,https://revistasinvestigacion.unmsm.edu.pe/ind...,2025-02-20
1,RECOLECTA___::24cb4438d1afe299e63cfdea4a31911f,http://hdl.handle.net/10261/97737,2025-02-20
2,RECOLECTA___::24cb4438d1afe299e63cfdea4a31911f,http://hdl.handle.net/10261/97737,2025-02-20
3,core_ac_uk__::a391293fe8bd6c9c6c203ee15e8c2e8f,https://nottingham-repository.worktribe.com/fi...,2025-02-20
4,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,http://sedici.unlp.edu.ar/handle/10915/154923,2025-02-20
...,...,...,...
669,dedup_wf_002::0152976d6d89c7781aa7447192d99498,https://doaj.org/article/637f323d009e4f64be0e3...,2025-02-20
670,dedup_wf_002::0152976d6d89c7781aa7447192d99498,https://doaj.org/article/9ef82a10e3334a8ea5deb...,2025-02-20
671,dedup_wf_002::0152976d6d89c7781aa7447192d99498,http://sedici.unlp.edu.ar/handle/10915/41280,2025-02-20
672,dedup_wf_002::0152976d6d89c7781aa7447192d99498,None,2025-02-20
